# Validation of Nigeria model

This notebooks investigates the results obtained by pypsa-africa model for the Nigeria case.
In particular, public information of the power system of Nigeria are compared to those used and obtained from the pypsa-africa model.

The following quantities will be reviewed:
- inputs used by the pypsa-model:
  - network characteristics
  - substations
  - installed generation by type
- outputs of the simulation
  - demand
  - energy production shares

To properly reproduce the findings obtained in this notebook, please run the full snakemake procedure on the Nigeria case only.
To do so, please set ``countries = ["NG"]`` in the ``config.yaml`` file.

## Preparation

### Set main directory to root folder

In [1]:
# set current folders

import sys

sys.path.append("../../")  # to import helpers

from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")

This is the repository path:  /data/davidef/gitdf/pypsa-africa
Had to go 2 folder(s) up.


### Definition of main folder paths

In [28]:
import os

# OSM raw data files
substations_OSMraw_path = os.getcwd() + "/data/raw/africa_all_raw_substations.geojson"
lines_OSMraw_path = os.getcwd() + "/data/raw/africa_all_raw_lines.geojson"

# cleaned osm data files
substations_OSMclean_path = os.getcwd() + "/data/clean/africa_all_substations.geojson"
lines_OSMclean_path = os.getcwd() + "/data/clean/africa_all_lines.geojson"

# shapes files
countries_shape_path = os.getcwd() + "/resources/country_shapes.geojson"

# solve network output
solved_network_path = os.getcwd() + "/results/networks/elec_s_100_ec_lcopt_Co2L-3H.nc"

### Import packages

In [3]:
# import packages

import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature

from scipy.sparse import csgraph
from itertools import product
import re

from shapely.geometry import Point, LineString
import shapely, shapely.prepared
from shapely.wkt import loads
from shapely.validation import make_valid

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)

## 1. Analysis of network of the Nigeria case

In this subsection, the data publicy on the network power system are compared. In particular, the main data that are being compared are:
- Layout of the network as shown by images (Section 1.1)
- Total length of the line (Section 1.2)

The data used for the comparison have been taken from public reliable sources, with strong focus on the website of the Nigerian Transmission operator: Nigerian Transmission Company ([TCN](https://www.tcn.org.ng/)).

In the Section 1.1, the network layout obtained by the workflow is drawn to be possibly compared with the image as shown in the public website of the [Nigerian Transmission System Operator](https://www.tcn.org.ng/). The image below, dated 2016, available from the [Nigerian Transmission System Operator](https://www.tcn.org.ng/), depicts the national power grid of Nigeria.

![Nigerian transmission system](../../images/Nigeria_data/Nigeria_network_map.png)
[Source link](https://www.tcn.org.ng/repository/maps/GRID%20MAP%20FOR%20EXISTING%20TRANSMISSION%20AND%20GENERATION%20NETWORK.pdf)

In the Section 1.2, the network length as obtained from the workflow is calculated to then be compared with the information reported by the Nigerian TSO. According to the data available from the [Transmission Company of Nigeria (ISO)](https://nerc.gov.ng/index.php/home/nesi/404-transmission), in Nigeria, there are:

- \> 20000km lines
- \> 7500MW line capacity

These information shall be close to those obtained by the workflow.

### 1.1. Network layout (using OSM clean data)

In this section, the cleaned OSM data are used to draw plots of the entire network, to reproduce the image available from the Nigerian TSO and verify the quality of the pypsa-africa model.

#### 1.1.1. Load clean OSM data

In [4]:
# load substation geodataframe
df_substations_osm_clean = gpd.read_file(
    substations_OSMclean_path
)  # .to_crs(epsg=3857)
# df_substations_osm_clean = df_substations_osm_clean[
#     df_substations_osm_clean.country == "NG"
# ]
df_substations_osm_clean["geometry"] = df_substations_osm_clean["geometry"].apply(
    make_valid
)
# load lines geodataframe
df_lines_osm_clean = gpd.read_file(lines_OSMclean_path)  # .to_crs(epsg=3857)
# df_lines_osm_clean = df_lines_osm_clean[df_lines_osm_clean.country == "NG"]
df_lines_osm_clean["geometry"] = df_lines_osm_clean["geometry"].apply(make_valid)

#### 1.1.2. Calculate voltage levels for substations and lines

Substations voltage level [V]

In [5]:
voltage_substations = df_substations_osm_clean.voltage.unique()
voltage_substations.sort()
voltage_substations

array([ 44000,  45000,  60000,  63000,  66000,  69000,  70000,  88000,
        90000, 110000, 115000, 120000, 132000, 138000, 150000, 161000,
       220000, 222000, 225000, 230000, 275000, 330000, 350000, 380000,
       400000, 500000, 533000, 765000])

Lines voltage level [V]

In [6]:
unique_voltages = df_lines_osm_clean.voltage.unique()
unique_voltages.sort()
unique_voltages

array([ 44000,  45000,  60000,  63000,  66000,  69000,  70000,  88000,
        90000, 110000, 115000, 120000, 132000, 138000, 150000, 161000,
       220000, 222000, 225000, 230000, 275000, 330000, 350000, 380000,
       400000, 500000, 533000, 765000])

#### 1.1.3. Specify colors for voltage levels in agreement with the image by the TSO

Specify color array by voltage level

In [7]:
color_voltages = ["orange", "blue", "red"]

voltage_to_color = {v: c for (v, c) in zip(unique_voltages, color_voltages)}
voltage_to_color

{44000: 'orange', 45000: 'blue', 60000: 'red'}

#### 1.1.4. Plot the network data

Get the borders of the country shape to perform the desired plot

In [9]:
# get all country shapes
country_shapes = gpd.read_file(countries_shape_path)

# get nigeria shape
nigeria_shape = country_shapes.set_index("name").geometry

# get bounds
print("Original bounds:")
print(nigeria_shape.boundary.bounds)

# add tolerance to bounds
tol = 0.2

bounds_mod = list(nigeria_shape.boundary.bounds)
bounds_mod[0] -= tol  # reduce minx
bounds_mod[1] -= tol  # reduce miny
bounds_mod[2] += tol  # increase maxx
bounds_mod[3] += tol  # increase maxy

print("Modified bounds:")
print(bounds_mod)

# reorder bounds to comply with extend function (x0, x1, y0, y1)
extent_list = (bounds_mod[0], bounds_mod[2], bounds_mod[1], bounds_mod[3])

Original bounds:
           minx       miny       maxx       maxy
name                                            
AO    11.671251 -18.041758  24.082117  -4.377671
BI    29.001749  -4.470001  30.850225  -2.309835
NE     0.166250  11.696970  15.995642  23.525030
NA    11.734862 -28.969450  25.253639 -16.959890
MG    43.219860 -25.605694  50.486526 -11.952085
CG    11.200858  -5.030664  18.650000   3.703082
EH   -17.105415  20.769583  -8.670013  27.683098
ER    36.438766  12.357313  43.137638  18.006536
DJ    41.760452  10.952411  43.417641  12.706778
CD    12.206629 -13.455676  31.305704   5.385852
MR   -17.066521  14.715551  -4.827674  27.298071
EG    24.698099  21.725389  36.215137  31.667360
CF    14.417741   2.220515  27.463421  11.007570
GW   -16.714891  10.921250 -13.636520  12.684263
ML   -12.238912  10.161430   4.242887  25.000000
ZA    16.451891 -34.832916  32.891251 -22.125111
MZ    30.217415 -26.868694  40.837360 -10.471250
ZW    25.237177 -22.420277  33.055019 -15.609359
CI 

TypeError: unsupported operand type(s) for -=: 'str' and 'float'

Draw the picture using pyplot

In [12]:
%matplotlib inline

# get the structure of the background data to plot
imagery = OSM()

max_width = 30  # max width of the figure
max_height = 30  # max heifht of the figure

# calculate figure size with appropriate multiplier to adhere to the desired width/height
multiplier = min(
    max_width / (extent_list[1] - extent_list[0]),
    max_height / (extent_list[3] - extent_list[2]),
)
figsize = (
    (extent_list[1] - extent_list[0]) * multiplier,
    (extent_list[3] - extent_list[2]) * multiplier,
)


fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(1, 1, 1, projection=imagery.crs)  # specify projection
ax.set_extent(extent_list, ccrs.PlateCarree())  # specify the location of the image
ax.add_image(imagery, 7)  # add the background image

# create an auxiliary dataframe for the substations with the desired crs and color properties
df_substations_osm_clean_plot = df_substations_osm_clean.to_crs(imagery.crs)
# specify the color of the nodes
df_substations_osm_clean_plot["color"] = df_substations_osm_clean_plot.voltage.apply(
    lambda x: voltage_to_color[x]
)


# create an auxiliary dataframe for the lines with the desired crs and properties
df_lines_osm_clean_plot = df_lines_osm_clean.to_crs(imagery.crs)
df_lines_osm_clean_plot[
    "centroids"
] = df_lines_osm_clean_plot.geometry.boundary.centroid  # get the centroids of the line
df_lines_osm_clean_plot["color"] = df_lines_osm_clean_plot.voltage.apply(
    lambda x: voltage_to_color[x]
)  # specify the color of the line

# draw the substations
df_substations_osm_clean_plot.plot(color=df_substations_osm_clean_plot.color, ax=ax)

# draw the lines
df_lines_osm_clean_plot.plot(color=df_lines_osm_clean_plot.color, ax=ax)

# add annotations to show the number of circuits by line
for (id, row) in df_lines_osm_clean_plot.iterrows():
    ax.text(row.centroids.x, row.centroids.y, row.circuits, color=row.color)

NameError: name 'extent_list' is not defined

Add the equivalent map but with interactive features

In [ ]:
# Disabled to ease the data storing in github; decomment to use

# df_substations_osm_clean[df_substations_osm_clean.voltage == unique_voltages[0]].hvplot(
#     geo=True,
#     size = 10,  # buses["tag_area"]**(0.5)/10,
#     frame_height=750,
#     alpha=0.4,
#     tiles='CartoLight',
#     color=color_voltages[0],
#     hover_cols=['bus_id'],
# ) * df_substations_osm_clean[df_substations_osm_clean.voltage == unique_voltages[1]].hvplot(
#     geo=True,
#     alpha=0.4,
#     color=color_voltages[1],
#     hover_cols=['bus_id'],
# ) * df_substations_osm_clean[df_substations_osm_clean.voltage == unique_voltages[2]].hvplot(
#     geo=True,
#     alpha=0.4,
#     color=color_voltages[2],
#     hover_cols=['bus_id'],
# ) * df_lines_osm_clean[df_lines_osm_clean.voltage == unique_voltages[0]].hvplot(
#     geo=True,
#     color=color_voltages[0],
#     alpha=0.4,
#     hover_cols=['circuits', 'line_id'],
# ) * df_lines_osm_clean[df_lines_osm_clean.voltage == unique_voltages[1]].hvplot(
#     geo=True,
#     color=color_voltages[1],
#     alpha=0.4,
#     hover_cols=['circuits', 'line_id'],
# ) * df_lines_osm_clean[df_lines_osm_clean.voltage == unique_voltages[2]].hvplot(
#     geo=True,
#     color=color_voltages[2],
#     alpha=0.4,
#     hover_cols=['circuits', 'line_id'],
# ).opts(
#     active_tools=['pan', 'wheel_zoom']
# )

#### 1.1.5. Comments on the network layout

By comparing the image of the existing lines availabe from TCN and the plots obtained from the workflow, it is possible to notice that they overlap and there are only minimal differences.

However, the available source is dated 2016 and, as stated by the same [TCN source](https://www.tcn.org.ng/repository/maps/GRID%20MAP%20FOR%20EXISTING%20TRANSMISSION%20AND%20GENERATION%20NETWORK.pdf), there where several transmission lines projects ongoing in 2015.
Therefore, the current model available in OSM may be sligthly outdated, but it draws a good picture of the current situation, given the available sources.

### 1.2. Calculate the total length of the lines

In this section, the total length of the lines are obtained. To do so, the following actions are performed:
1. Preliminary data loading
2. Crosscheck the unit of measurement to verify that the units are in meters [m]
3. Crosscheck the number of circuits for geometry line in the raw OSM dataset
4. Evaluate the total length of the lines in the raw OSM dataset
5. Evaluate the total length of the lines in the clean OSM dataset
6. Remarks on the line length

Note: the OSM dataset reports the geometry paths of the lines; however, in the same geometry path there may multiple circuits. That information is contained in the number of cables for geometry line, which is the reason why this information is properly addressed in point 3 (Subsection 1.2.3).

#### 1.2.1 Preliminary data loading of raw OSM data

In the following, we show the input data downloaded from OSM; the clean data have already been loaded.

In [13]:
# load substation geodataframe
df_substations_osm_raw = gpd.read_file(substations_OSMraw_path)
# df_substations_osm_raw = df_substations_osm_raw[df_substations_osm_raw.Country == "NG"]
# load lines geodataframe
df_lines_osm_raw = gpd.read_file(lines_OSMraw_path)
# df_lines_osm_raw = df_lines_osm_raw[df_lines_osm_raw.Country == "NG"]

# note that epsg=3857 has been chosen so that lenghts are in meters

#### 1.2.2 Crosscheck the unit of measurement

In the following, we verify that the property ``geometry.length`` can be successfully used to measure the length of the line in meters (spoiler, it is). To do so, we need to use the crs ``EPSF:3763`` and compare the obtained value with the column ``Length`` available in the raw OSM data of dataframe ``df_lines_osm_raw``.

As shown below, the two informations are very close which confirm the proposed approach.

In [14]:
df_lines_osm_raw["length_by_geometry"] = df_lines_osm_raw.to_crs(
    epsg=3857
).geometry.length
df_lines_osm_raw[["Length", "length_by_geometry"]].head()

,Length,length_by_geometry
0,11510.599530,11510.599530
1,54.334966,54.334966
2,108.331636,108.331636
3,48.181889,48.181889
4,56.434628,56.434628


#### 1.2.3 Verify the value of column ``tags.cables``

In the following we show the value of the dataframe column ``tags.cables`` to validate and verify the quality of the approximation used to estimate the total length.
This property describes the number of cables per geometric line, as reported in the OSM dataset and it is an important information to estimate the total length of the lines operated by each TSO.
In fact each geometric path may enable to carry multiple circuits, thus it is important to properly account to them when it occurs. Sicne three phase lines are mostly used, each circuit is composed by three cables.

In particular, we show that only integer and None values are available: by convention the None values will be changed to 3 as base-case scenario.

In [15]:
df_lines_osm_raw["tags.cables"].unique()

array(['6', '3', None, '2', '4', '1', '5', '12', '9', '7', '0'],
      dtype=object)

#### 1.2.4. Calculate the total line length for OSM raw data

In this case, we calculate the total length of the lines.

Show line length by voltage level [km]

Note: when multiple circuits are available, the length of the line is multiplied accordingly.

In [16]:
# Note, since CRS EPSG:3763 is used, distances are in meters, thus by dividing for 1000, the units are in km
df_lines_osm_raw["cables_fix"] = df_lines_osm_raw["tags.cables"].copy()
df_lines_osm_raw.loc[
    df_lines_osm_raw["tags.cables"].isnull() | df_lines_osm_raw["tags.cables"].isna(),
    "cables_fix",
] = 3  # when NaN or None, set default value
df_lines_osm_raw["length"] = (
    df_lines_osm_raw.to_crs(epsg=3857).geometry.length
    * df_lines_osm_raw["cables_fix"].astype(float)
    / 3
)
df_lines_osm_raw.groupby(by=["tags.voltage"]).length.sum() / 1000

tags.voltage
0                        101.143805
11                        20.673653
11000                     38.003393
110000                  7451.601511
115000                    22.489348
                           ...     
Avenida La Esperanza       2.287838
KV30                       2.706811
high-medium                5.444692
low                        0.953750
medium                   147.618223
Name: length, Length: 67, dtype: float64

Total lines length [km]

In [17]:
df_lines_osm_raw.to_crs(epsg=3857).length.sum() / 1000

413196.7544242994

Furthermore, for the purpose of clarity, it is also shown the total length of the line where information on the number of circuits has been fixed (length of the lines where the cables data where missing)

Show total line length [km] for lines where information on the number of circuits is not available

In [18]:
# Note, since CRS EPSG:3763 is used, distances are in meters, thus by dividing for 1000, the units are in km
df_lines_osm_raw.to_crs(epsg=3857)[
    df_lines_osm_raw["tags.cables"].isnull()
    | df_lines_osm_raw["tags.cables"].isna()  # when NaN or None, set default value
].geometry.length.sum() / 1000

61828.24251412215

#### 1.2.5. Calculate the total line length for OSM clean data

Finally, the information on total line length for the clean dataset are calculated.

Show line length by voltage level [km]

Note: when multiple circuits are available, the length of the line is multiplied accordingly.

In [19]:
# Note, since CRS EPSG:3763 is used, distances are in meters, thus by dividing for 1000, the units are in km
df_lines_osm_clean["length"] = (
    df_lines_osm_clean.to_crs(epsg=3857).geometry.length
    * df_lines_osm_clean.circuits
    / 1000
)
df_lines_osm_clean.groupby(by=["voltage"]).length.sum()

voltage
44000         60.327569
45000         31.882627
60000      34534.258248
63000        421.352794
66000      41484.542030
69000         50.816354
70000        336.221484
88000      11404.255999
90000       5990.580169
110000      7455.676984
115000        22.489348
120000      1184.261679
132000     68056.526768
138000       147.214901
150000      4479.662794
161000      6980.942528
220000    100140.024842
222000        13.391679
225000     41727.977947
230000      7856.891480
275000      8834.852446
330000     21091.489842
350000      4028.509825
380000         0.295992
400000     58279.628955
500000     17459.497205
533000      4783.407812
765000      3216.830039
Name: length, dtype: float64

Total lines length [km]

In [20]:
df_lines_osm_clean.to_crs(epsg=3857).length.sum() / 1000

371516.19283394335

In [21]:
df_lines_osm_clean["length"] = (
    df_lines_osm_clean.to_crs(epsg=3857).geometry.length
    * df_lines_osm_clean.circuits
    / 1000
)
df_lines_osm_clean.groupby(by=["voltage"]).length.sum()

voltage
44000         60.327569
45000         31.882627
60000      34534.258248
63000        421.352794
66000      41484.542030
69000         50.816354
70000        336.221484
88000      11404.255999
90000       5990.580169
110000      7455.676984
115000        22.489348
120000      1184.261679
132000     68056.526768
138000       147.214901
150000      4479.662794
161000      6980.942528
220000    100140.024842
222000        13.391679
225000     41727.977947
230000      7856.891480
275000      8834.852446
330000     21091.489842
350000      4028.509825
380000         0.295992
400000     58279.628955
500000     17459.497205
533000      4783.407812
765000      3216.830039
Name: length, dtype: float64

#### 1.2.6. Remarks on the line length

According to the data available from the [Transmission Company of Nigeria (ISO)](https://nerc.gov.ng/index.php/home/nesi/404-transmission), in Nigeria, there are:

- \> 20000km lines
- \> 7500MW line capacity

Therefore, the current OSM data cleaning captures around 60% (12752/20000) the network length; yet, this value is pretty close to the input data available from OSM data.

The shape, however, is quire representative of the network shown in the image above. A big contribution to solve this element may be related to the number of circuits that may not be well corresponding to what actually in place. About 2800 km of geometric lines distances do not have the number of circuits specified.

Moreover, recent network upgrades may not be well represented.

## 2. Analysis on the demand data

In the following section, let's investigate the electrical demand used in the simulations.

According to [IEA](https://www.iea.org/articles/nigeria-energy-outlook), the electricity demand in Nigeria was about 57 TWh.

Let's verify the input used in pypsa-africa.

#### Load the model

In [29]:
n_solve = pypsa.Network(solved_network_path)

INFO:pypsa.io:Imported network elec_s_100_ec_lcopt_Co2L-3H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


#### Calculate total demand

Calculate the total demand used in the simulations.
Please, note that the actual demand shall be scaled to account for the number of representative days/periods that have been used.

In [23]:
# filter only Nigerian demand
nigeria_demand = n_solve.loads_t.p.filter(regex="NG *", axis=1)

# raw sum used in the model
tot_yearly_demand_scaled = nigeria_demand.sum().sum()

# rescaled value accounting for the number of representative days: the factor (8760/nigeria_demand.shape[0]) is the multiplier
tot_yearly_demand = tot_yearly_demand_scaled * 8760 / nigeria_demand.shape[0]

# show value in TWh
print("Demand in TWh: %.2f" % (tot_yearly_demand / 1000000))

Demand in TWh: 0.00


#### Comments on the demand

It is worth noticing that the expected demand is aligned to the current Nigerian electrical demand

## 3. Installed generation by type

In the subsequent section, the total installed generation is compared to validate the model.

According to [USAID](https://www.usaid.gov/powerafrica/nigeria), the total installed generation in Nigeria is:
- Hydro: 2.062 GW
- Gas: 11.972 GW
- Wind: 0.01 MW
- Solar: 0.007 MW
- Other/Diesel/HFO: 2.333 GW

Total: 16.4 GW

These numbers are compared to the ones used in pypsa-africa

In [30]:
n_solve.generators

,p_nom_max,weight,p_nom,capital_cost,carrier,marginal_cost,bus,p_nom_extendable,efficiency,control,p_nom_opt,sign,type,p_nom_min,p_min_pu,p_max_pu,p_set,q_set,build_year,lifetime,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AO 0 CCGT,NaN,0.000000,1.636800e+02,0.000000,CCGT,47.209399,AO 0,False,0.500,Slack,NaN,1.000,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
AO 0 oil,NaN,0.000000,3.212408e+02,0.000000,oil,130.236478,AO 0,False,0.393,,NaN,1.000,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
AO 0 onwind,2.444418e+06,147268.336082,0.000000e+00,109295.569252,onwind,0.024083,AO 0,True,1.000,,NaN,1.000,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
AO 0 ror,NaN,362.038729,3.620387e+02,270940.715283,ror,0.009533,AO 0,False,0.900,,NaN,1.000,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
AO 0 solar,6.400803e+05,77387.019006,0.000000e+00,55064.071419,solar,0.020896,AO 0,True,1.000,,NaN,1.000,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZA 7 load,inf,NaN,1.000000e+09,0.000000,load,8000.009034,ZA 7,False,1.000,PQ,0.0,0.001,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
ZA 8 load,inf,NaN,1.000000e+09,0.000000,load,8000.009850,ZA 8,False,1.000,PQ,0.0,0.001,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
ZA 9 load,inf,NaN,1.000000e+09,0.000000,load,8000.010883,ZA 9,False,1.000,PQ,0.0,0.001,,0.0,0.0,1.0,0.0,0.0,0,inf,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0


In [33]:
# filter only generators in Nigeria
nigeria_generators = n_solve.generators  # .filter(regex="NG *", axis=0)

# obtain installed generation by carrier
nigeria_generators.groupby(by="carrier")["p_nom"].sum()

carrier
CCGT      5.452027e+04
OCGT      1.376421e+04
coal      4.104036e+04
load      1.000000e+11
oil       4.986082e+04
onwind    0.000000e+00
ror       6.264847e+03
solar     0.000000e+00
Name: p_nom, dtype: float64

The current data are of the same order of magnitude of those declared by public authorities, yet up-to-date data shall be used.

## 4. Energy production shares

In the following, the total energy production shares as output of the total energy dispatch as available by IEA are compared to those calculated by the pypsa-africa procedure.

The following image depicts the energy demand by source as computed in the Stated Policy Scenarios by [IEA](https://www.iea.org/articles/nigeria-energy-outlook), which is used as a reference for comparison purposes

![Nigerian demand - IEA](../../images/Nigeria_data/Nigeria_demand.png)
[Source link](https://www.iea.org/articles/nigeria-energy-outlook)

In [32]:
# filter only generators in Nigeria
nigeria_generators_dispatch = n_solve.generators_t.p.filter(regex="NG *", axis=1).copy()

# rename column to keep only the carrier
nigeria_generators_dispatch.rename(
    columns=dict(
        zip(
            nigeria_generators_dispatch.columns,
            map(
                lambda x: re.sub(r"NG \d+ ", "", x), nigeria_generators_dispatch.columns
            ),
        )
    ),
    inplace=True,
)

# obtain generation by carrier
nigeria_generators_dispatch.groupby(
    level=0, axis=1
).sum().sum() * 8760 / nigeria_generators_dispatch.shape[0] / 1000000

Series([], dtype: float64)

#### Comments
- Currently Oil (back-up generators) is not dispatched and numbers do not match
- hydro is basically missing